Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [24]:
df_ratings = pd.read_csv('https://github.com/Kitchess/Netology/blob/master/hw_14/ratings.csv')

df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [33]:
counted_ratings = df_ratings.groupby('userId', as_index=False).agg({'rating' : 'count'})

counted_ratings

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100
...,...,...
666,667,68
667,668,20
668,669,37
669,670,31


In [34]:
counted_ratings_over_100 = counted_ratings.loc[(counted_ratings.rating > 100)]

counted_ratings_over_100

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423
...,...,...
655,656,128
658,659,142
663,664,519
664,665,434


In [14]:
df_timestamps = df_ratings.groupby('userId', as_index=False)\
.agg(timestamp_min = ('timestamp', 'min'), timestamp_max = ('timestamp', 'max'))

df_timestamps

,userId,timestamp_min,timestamp_max
0,1,1260759108,1260759205
1,2,835355395,835356246
2,3,1298861589,1298932787
3,4,949778714,949982274
4,5,1163373044,1163375145
...,...,...,...
666,667,847271220,847272234
667,668,993613196,993613478
668,669,1015829081,1015829766
669,670,938781328,940944033


In [27]:
df_timestamps['lifetime'] = df_timestamps['timestamp_max'] - df_timestamps['timestamp_min']

df_timestamps

,userId,timestamp_min,timestamp_max,lifetime
0,1,1260759108,1260759205,97
1,2,835355395,835356246,851
2,3,1298861589,1298932787,71198
3,4,949778714,949982274,203560
4,5,1163373044,1163375145,2101
...,...,...,...,...
666,667,847271220,847272234,1014
667,668,993613196,993613478,282
668,669,1015829081,1015829766,685
669,670,938781328,940944033,2162705


In [29]:
lifetime_over_100 = counted_ratings_over_100.merge(df_timestamps, how='inner')

lifetime_over_100

,userId,rating,timestamp_min,timestamp_max,lifetime
0,4,204,949778714,949982274,203560
1,8,116,1154389340,1154474527,85187
2,15,1700,997937239,1469330735,471393496
3,17,363,1127468587,1127476640,8053
4,19,423,855190091,855195373,5282
...,...,...,...,...,...
253,656,128,986240991,986244044,3053
254,659,142,834598040,866207451,31609411
255,664,519,1343731283,1441911722,98180439
256,665,434,992836298,1046967769,54131471


In [35]:
lifetime_mean = lifetime_over_100['lifetime'].mean()

lifetime_mean

40080507.4496124

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd - железнодорожные перевозки

auto - автомобильные перевозки

air - воздушные перевозки

client_base - адреса клиентов

In [36]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [37]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [38]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [39]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [51]:
merged = rzd.merge(auto, how='outer').merge(air, how='outer')

merged

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [52]:
address_merged = merged.merge(client_base, how='left')

address_merged

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Ответ:

Чтобы добавить данные по местоположению в таблицу визитов и покупок необходимо использовать метод .merge(how=left), тогда все наборы координат войдут в таблицу. А если соответсвия в таблице purchase не будет, то данные о покупках все равно не потеряются.


In [21]:
coordinates = pd.DataFrame(
    {
        'user_id': [111, 222, 111, 444, 555],
        'coordinate': ['55.755831 37.617673', '56.755831 37.617673', '55.755831 38.617673', '55.755831 37.617673', '55.255831 37.017673'],
    }
)


In [24]:
purchase = pd.DataFrame(
    {
        'user_id': [111, 222, 333, 444, 555, 666],
        'source': ['other', 'organic', 'email', 'organic', 'other', 'other' ],
        'category': ['Продукты', 'Электроника', 'Продукты', 'Одежда', 'Досуг', 'Бытовая техника']
    }
)

purchase

,user_id,source,category
0,111,other,Продукты
1,222,organic,Электроника
2,333,email,Продукты
3,444,organic,Одежда
4,555,other,Досуг
5,666,other,Бытовая техника


In [25]:
table = purchase.merge(coordinates, how='left')

table

,user_id,source,category,coordinate
0,111,other,Продукты,55.755831 37.617673
1,111,other,Продукты,55.755831 38.617673
2,222,organic,Электроника,56.755831 37.617673
3,333,email,Продукты,NaN
4,444,organic,Одежда,55.755831 37.617673
5,555,other,Досуг,55.255831 37.017673
6,666,other,Бытовая техника,NaN
